# Aim
Check whether DECD param give consistent fidelities with Qutip Simulations

# Imports

In [1]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')


In [2]:
# for param file 
#%%
# note: timestamp can't use "/" character for h5 saving.
TIMESTAMP_FORMAT = "%Y-%m-%d %H:%M:%S"
END_OPT_STRING = "\n" + "=" * 60 + "\n"
import numpy as np
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # supress warnings
import h5py

# print(
#     "\nNeed tf version 2.3.0 or later. Using tensorflow version: "
#     + tf.__version__
#     + "\n"
# )
import ECD_control.ECD_optimization.tf_quantum as tfq
from ECD_control.ECD_optimization.visualization import VisualizationMixin
import qutip as qt
import datetime
import time

In [3]:
# for pulse file
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
from scipy.signal import find_peaks
from scipy.optimize import fmin



# Finding Param

In [4]:
from DECD_param import FixedBatchOptimizer as BO

In [6]:
#The initial oscillator state.
N1 =5
N2 =5
Fock1 = 0
Fock2= 0
psi_i1 = qt.basis(N1,Fock1) #initial state
psi_i2 = qt.basis(N2,Fock2)
psi_initial = qt.tensor(psi_i1, psi_i2)

#Target
Fock1 = 1
Fock2= 0
psi_t1 = qt.basis(N1,Fock1) #target state
psi_t2 = qt.basis(N2,Fock2)
psi_target = qt.tensor(psi_t1, psi_t2)
psi_target

#Optimization of ECD Circuit parameters (betas, gammas, phis, and thetas)
#the optimization options
opt_params = {
'N_blocks' : 4, #circuit depth
'N_multistart' : 2, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 100, #number of epochs before termination
'epoch_size' : 20, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.999, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'gamma_scale' : 3.0, #maximum |gamma| for random initialization
'N_cav1': N1, #number of levels in mode 1
'N_cav2': N2, #number of levels in mode 2
'initial_states' : [qt.tensor(qt.basis(2,0),psi_initial)], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_target)], #end in |e> |target>.
#"initial_params": init_params,
'name' : 'Fock1 %d' % Fock1, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}



In [7]:
opt = BO(**opt_params)
opt.print_info()

None
Fixed construct needed matrices called
optimization_type: state transfer
N_multistart: 2
N_blocks: 4
term_fid: 0.999
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 20
epochs: 100
beta_scale: 3.0
gamma_scale: 3.0
alpha1_scale: 1.0
alpha2_scale: 1.0
theta_scale: 3.141592653589793
use_etas: False
use_displacements: False
use_phase: False
name: Fock1 1
comment: 
initial_params: None
N_cav1: 5
N_cav2: 5
filename: Fock1 1.h5

Best circuit parameters found:
betas:         [-1.31239-0.67512j -0.57802+1.20912j -1.3443 -1.47278j -1.29727-2.4183j ]
gammas:         [-0.25604-0.06847j  0.14317-0.36955j  0.28456-2.66201j -0.94836+2.68794j]
alphas1:        [0.+0.j]
alphas2:        [0.+0.j]
phis (deg):    [   0.       162.7622  -130.12808  164.99426]
etas (deg):    [89.99999 89.99999 89.99999 89.99999]
thetas (deg):  [-145.14816  -16.57494 -127.16859  -44.87994]
Max Fidelity:  0.010137




In [8]:
opt.optimize()

Start time: 2022-05-23 09:02:47
 Epoch: 36 / 100 Max Fid: 0.999754 Avg Fid: 0.983569 Max dFid: 0.002104 Avg dFid: 0.001045 Elapsed time: 0:00:20.353043 Remaing time: 0:00:36.1831875

 Optimization stopped. Term fidelity reached.

optimization_type: state transfer
N_multistart: 2
N_blocks: 4
term_fid: 0.999
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 20
epochs: 100
beta_scale: 3.0
gamma_scale: 3.0
alpha1_scale: 1.0
alpha2_scale: 1.0
theta_scale: 3.141592653589793
use_etas: False
use_displacements: False
use_phase: False
name: Fock1 1
comment: 
initial_params: None
N_cav1: 5
N_cav2: 5
filename: Fock1 1.h5

Best circuit parameters found:
betas:         [-0.19928+3.45057j -0.0682 +1.22365j -2.06821-2.96092j  1.9319 +2.27804j]
gammas:         [-2.91335+0.00602j  3.09736-0.00664j -0.72343-1.02316j -0.91717-1.019j  ]
alphas1:        [0.+0.j]
alphas2:        [0.+0.j]
phis (deg):    [  0.      -34.68935  67.77032 -47.32071]
etas (deg):    [89.99999 89.99999 89.99999 89.999

'2022-05-23 09:02:47'

# Verifying with ECD single mode

In [9]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram\\ECD_control')


In [10]:
from ECD_control import ECD_optimization
from ECD_control import ECD_pulse_construction

import numpy as np
import qutip as qt 
from ECD_control.ECD_optimization.batch_optimizer import BatchOptimizer
from ECD_control.ECD_optimization.optimization_analysis import OptimizationAnalysis, OptimizationSweepsAnalysis
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


Need tf version 2.3.0 or later. Using tensorflow version: 2.7.0


Need tf version 2.3.0 or later. Using tensorflow version: 2.7.0



[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1937876853007653130
 xla_global_id: -1]

In [11]:
#The target oscillator state.
N =5
Fock = 1
psi_t = qt.basis(N,Fock) #target state

In [15]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
#the optimization options
opt_params = {
'N_blocks' : 4, #circuit depth
'N_multistart' : 2, #Batch size (number of circuit optimizations to run in parallel)
'epochs' : 200, #number of epochs before termination
'epoch_size' : 10, #number of adam steps per epoch
'learning_rate' : 0.01, #adam learning rate
'term_fid' : 0.995, #terminal fidelitiy
'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
'beta_scale' : 3.0, #maximum |beta| for random initialization
'initial_states' : [qt.tensor(qt.basis(2,0),qt.basis(N,0))], #qubit tensor oscillator, start in |g> |0>
'target_states' : [qt.tensor(qt.basis(2,0), psi_t)], #end in |e> |target>.
'name' : 'Fock %d' % Fock, #name for printing and saving
'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}


#note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
#in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
#So, we ask the optimizer to end in |e> |target> instead of |g>|target>.

In [16]:
ecd_opt = BatchOptimizer(**opt_params)

#print optimization info. 
ecd_opt.print_info()

optimization_type: state transfer
N_multistart: 2
N_blocks: 4
term_fid: 0.995
dfid_stop: 1e-06
no_CD_end: False
learning_rate: 0.01
epoch_size: 10
epochs: 200
beta_scale: 3.0
alpha_scale: 1.0
theta_scale: 3.141592653589793
use_displacements: False
use_phase: False
name: Fock 1
comment: 
N_cav: 5
filename: Fock 1.h5

Best circuit parameters found:
betas:         [ 0.86955-2.66811j  0.79405+1.13483j -0.02411+2.67525j -2.22024-0.01295j]
alphas:        [0.+0.j]
phis (deg):    [   0.      -105.39643 -155.45955   35.75822]
thetas (deg):  [  7.90277 -14.81734  65.12698 -95.96886]
Max Fidelity:  0.033947




In [17]:
decd_opt = opt
ecd_opt.batch_fidelities(decd_opt.betas_rho,
                decd_opt.betas_angle,
                decd_opt.alphas1_rho,
                decd_opt.alphas1_angle,
                decd_opt.phis,
                decd_opt.thetas,)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.9974985 , 0.95410556], dtype=float32)>

In [19]:
decd_opt.batch_fidelities(decd_opt.betas_rho,
                decd_opt.betas_angle,
                decd_opt.gammas_rho,
                decd_opt.gammas_angle,
                decd_opt.alphas1_rho,
                decd_opt.alphas1_angle,
                decd_opt.alphas2_rho,
                decd_opt.alphas2_angle,
                decd_opt.phis,
                decd_opt.etas,
                decd_opt.thetas,)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.99167275, 0.9673773 ], dtype=float32)>